In [1]:
# import public packages
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle
import scipy
import mne
import sys

from mne.time_frequency import tfr_morlet
from mne.baseline import rescale
from mne.stats import permutation_cluster_test
from scipy.signal import spectrogram, hann, butter, filtfilt, hilbert
from scipy import signal, interpolate, stats
from scipy.interpolate import make_interp_spline, BSpline
from io import open
from importlib import reload

# import own functions
from utils import find_folders
import dat_preproc
import fix_annot_onsets
import mat2fif
import baseline_correction
import normalization

### Add Directories/ Load Files

In [2]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT


### Select Peaks and Plot Analytic Signal

In [ ]:
#Load raw fif data

test_raw = mne.io.read_raw_fif(os.path.join(
    project_path,
        'data',
        'Fifs',
        'test',
        'Sub009_FIF.fif'
    )
)

subID = 'Sub009'
fft_name = str(subID) + '_'
print(fft_name)

In [ ]:
#Load Power Spectra 
%matplotlib qt
ps_zscored = pd.read_csv(os.path.join(
    project_path,
        'data',
        'power_spectra',
        'test',
        'Sub005_FFT_zscoredPS.csv'
))
plt.plot(ps_zscored['StimOff'], label = 'Med On - Stim Off')
plt.plot(ps_zscored['StimOn'], label  = 'Med On - Stim On')
plt.legend(loc='upper right')
plt.xlabel('Frequency [Hz]')
plt.ylabel('LFP Power [zscored]')
plt.xlim(50,90)
plt.ylim(-0.6, 1.3)


In [ ]:
plt.savefig('Sub017_Exemplary_ZscoredPS',dpi = 150)

In [ ]:
reload(dat_preproc)
x = test_raw.get_data() #getting only RSTN channel
x1 = x[0,:]

peakMed = 81
peakStim = 65

dat_ngam = dat_preproc.low_highpass_filter(x1, peakMed-2, peakMed+2) 
dat_subh = dat_preproc.low_highpass_filter(x1, peakStim-2, peakStim+2) 

datall = [dat_ngam, dat_subh] 
labels = ['Peak 81Hz','Peak 65Hz']

In [ ]:
def window_rms(a, window_size):
  a2 = np.power(a,2)
  window = np.ones(window_size)/float(window_size)
  return np.sqrt(np.convolve(a2, window, 'valid'))

In [ ]:
%matplotlib qt
sm_signal_np = np.empty(shape = (2, x1.shape[0] - 499))
sm_signal_np[:] = np.nan

fig, axes = plt.subplots(1, 1, figsize=(12, 5))
wintosmooth = 500

for idx, dat in enumerate(datall):
    hiltr = hilbert(dat)
    amplitude_envelope = np.abs(hiltr)
    zscore_sign = stats.zscore(np.squeeze(amplitude_envelope))

    sm_signal = window_rms(zscore_sign, wintosmooth)
    
    plt.plot(sm_signal, label = labels[idx])
    #plt.plot(np.arange(0,75000), amplitude_envelope, label = labels[idx]) 
    
    #axes[idx].axvline(26250, color = 'b', ls='--', lw=2, label = 'Stim On')
    #axes[idx].axvline(50250, color = 'g', ls='--', lw=2, label = 'Stim Off')
    plt.ylabel('Analytic Signal')
    plt.xlim([0, sm_signal.shape[0]])

    
    sm_signal_np[idx,:] = sm_signal

    #axes[idx].set_xticks(ticks = np.arange(0, 80000, 10000), labels = np.arange(0,320,40))
    plt.xlabel('Time [sec]')

    

plt.suptitle('Smoothing Window: 500 samples')
plt.legend(loc='upper right')

plt.show()


In [ ]:
x2 = x[4, :] 
sm_stim = window_rms(x2, wintosmooth)
sm_stim1 = (sm_stim)

In [ ]:
fig, ax1 = plt.subplots(figsize = (18,6))
#plt.rcParams['font.size'] = 10
ax2 = ax1.twinx()
for idx, dat in enumerate(sm_signal_np):
    ax1.plot(sm_signal_np[idx,:], label = labels[idx], lw = 2)
ax2.plot(sm_stim1, label = 'Stimulation', color = 'grey', ls='--', lw=3, alpha = 0.4)
ax1.legend()
ax1.set_ylabel('Z-scored Smoothed Analytic Signal')
ax2.set_ylabel('Stimulation Amplitude [mA]')
#ax2.set_yticks(np.arange(0.5, 2.5, 0.25))
#ax2.set_yticklabels(np.arange(0.25, 2.25, 0.25))
#ax1.set_xlim(0, sm_signal_np.shape[0])
ax1.set_xticks(np.arange(0, 100000, 20000))
ax1.set_xticklabels(np.arange(0, 400, 80))
ax1.set_xlabel('Time [sec]')
plt.title(str(subID))
plt.show()

In [ ]:
sm_analSignal = np.transpose(np.squeeze(np.array([[sm_signal_np[0]], [sm_signal_np[1]],[sm_stim1]])))
sm_analSignal_df = pd.DataFrame(sm_analSignal,
    columns = ['Spontan','StimOn','StimVec'],
    )
print(sm_analSignal_df)

In [ ]:
fft_fig = os.path.join(project_path, 'figures','FFts','test/')
fft_file = os.path.join(project_path, 'data','anal_signal/')

plt.savefig(str(fft_fig)+str(fft_name)+'sm_analSignal',dpi = 300)
sm_analSignal_df.to_csv(str(fft_file)+str(fft_name)+'sm_analSignal.csv')

### Crop Analytic Signal to Epochs of Interest

In [ ]:
sm_anal = pd.read_csv(os.path.join(
    project_path,
        'data',
        'anal_signal',
        'Sub029_sm_analSignal.csv'
))

print(sm_anal)

subID = 'Sub029'
fft_name = str(subID) + '_'
print(fft_name)

In [ ]:
%matplotlib qt

plt.plot(np.arange(0, sm_anal.shape[0]), sm_anal['Spontan'])
plt.plot(np.arange(0, sm_anal.shape[0]), sm_anal['StimOn'])
plt.plot(np.arange(0, sm_anal.shape[0]), sm_anal['StimVec'], ls='--')


In [ ]:
#CROP IT

med1 = 0
med2 = 30
sub_pt = 34750
time_on = sub_pt - 25*250
time_off = sub_pt + 15*250

sm_anal1 = sm_anal.iloc[med1*250 : med2*250]
sm_anal2 = sm_anal.iloc[time_on : time_off]
sm_toplot = pd.concat([sm_anal1, sm_anal2])
print(sm_toplot)

In [ ]:
# CROP ONLY THE TRANSITION E.G. BETWEEN 0.5 - 1.0mA
this_sig = sm_toplot

fig, ax1 = plt.subplots(figsize = (8,6))
ax2 = ax1.twinx()

ax1.plot(np.arange(0, this_sig.shape[0]), this_sig['Spontan'], label = 'Med-Induced')
ax1.plot(np.arange(0, this_sig.shape[0]), this_sig['StimOn'], label = 'Subharmonic')
ax2.plot(np.arange(0, this_sig.shape[0]), this_sig['StimVec'], ls='--', color = 'grey')
ax1.axvline(x = 13750, color = 'grey', ls='--', lw=3, alpha = 0.4, label = 'Subharmonic On')
ax1.axvline(x = 7500, color = 'grey', lw=3, alpha = 0.2)


ax1.legend()
ax1.set_ylabel('Z-scored Smoothed Analytic Signal')
ax2.set_ylabel('Stimulation Amplitude [mA]')
ax1.set_xlabel('Time [samples]')
ax1.set_title(str(subID))


In [ ]:
fft_fig = os.path.join(project_path, 'figures','anal_signal/')
fft_file = os.path.join(project_path, 'data','anal_signal/')

plt.savefig(str(fft_fig)+str(fft_name)+'sm_analSignalTRANSITION',dpi = 150)
this_sig.to_csv(str(fft_file)+str(fft_name)+'sm_analSignalTRANSITION.csv')

In [ ]:
#PLOT IT

%matplotlib qt
plt.plot(np.arange(0,sm_toplot.shape[0]),sm_toplot['Spontan'], label = 'Peak 82Hz')
plt.plot(np.arange(0,sm_toplot.shape[0]),sm_toplot['StimOn'], label = 'Peak 72Hz')
plt.axvline(x = 7500, color = 'grey', ls='--', lw=3, alpha = 0.4, label = 'Stim On')
plt.xlabel('Time [samples]')
plt.ylabel('Z-scored Smoothed Analytic Signal')
plt.title(str(subID))
plt.xlim([0, sm_toplot.shape[0]])
plt.legend()
plt.show()

In [ ]:
fft_fig = os.path.join(project_path, 'figures','FFts','test/')
fft_file = os.path.join(project_path, 'data','anal_signal/')

plt.savefig(str(fft_fig)+str(fft_name)+'sm_analSignalCROPPED',dpi = 150)
sm_toplot.to_csv(str(fft_file)+str(fft_name)+'sm_analSignalCROPPED.csv')

### Average Cropped Analytic Signal and Plot

In [3]:
med_anal = pd.read_csv(os.path.join(
    project_path,
    'data',
    'anal_signal',
    'AllMed_analSignalTRANSITION.csv'
))

stim_anal = pd.read_csv(os.path.join(
    project_path,
    'data',
    'anal_signal',
    'AllStim_analSignalTRANSITION.csv'
))

print(stim_anal)

         sub005    sub006    sub009    sub014    sub017    sub029
0      0.796159  0.881777  0.851086  0.526401  0.806174  0.769114
1      0.796054  0.879257  0.848726  0.527014  0.804659  0.767466
2      0.796050  0.876840  0.846447  0.527712  0.804158  0.766451
3      0.796008  0.874098  0.844113  0.528500  0.803850  0.765561
4      0.795919  0.871046  0.841790  0.529381  0.803369  0.764568
...         ...       ...       ...       ...       ...       ...
17495  0.509684  1.596356  2.327574  2.518225  0.707037  1.090980
17496  0.510192  1.597226  2.321686  2.517199  0.707730  1.091854
17497  0.510687  1.598283  2.316014  2.516238  0.708463  1.092673
17498  0.511173  1.599522  2.310550  2.515339  0.709218  1.093440
17499  0.511650  1.600936  2.305284  2.514498  0.709975  1.094153

[17500 rows x 6 columns]


In [4]:
med_anal_mean = np.nanmean(med_anal,1)
med_anal_sem = stats.sem(med_anal,1, nan_policy='omit')

stim_anal_mean = np.nanmean(stim_anal,1)
stim_anal_sem = stats.sem(stim_anal,1, nan_policy='omit')


In [5]:
%matplotlib qt
plt.plot(np.arange(0,17500), med_anal_mean, label = 'Med-Induced FTG')
plt.fill_between(np.arange(0,17500), med_anal_mean-med_anal_sem, med_anal_mean+med_anal_sem, alpha = 0.2)

plt.plot(np.arange(0,17500 ), stim_anal_mean, color = 'red', label = 'Stim-Induced FTG')
plt.fill_between(np.arange(0,17500), stim_anal_mean-stim_anal_sem, stim_anal_mean+stim_anal_sem, color = 'red', alpha = 0.2)


plt.axvline(x = 13750, color = 'grey', ls='--', lw=3, alpha = 0.4, label = 'Subharmonic On')
plt.axvline(x = 7500, color = 'grey', lw=3, alpha = 0.2)

plt.legend()
plt.xlabel('Time [samples]')
plt.ylabel('Z-scored Smoothed Analytic Signal')

Text(0, 0.5, 'Z-scored Smoothed Analytic Signal')

In [8]:
fft_fig

'C:Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\ENTRAINMENT_PROJECT\\results'

In [7]:
fft_fig = os.path.join(project_path, 'results')

plt.savefig(str(fft_fig)+'AVG_AnalCropped',dpi = 200)

In [ ]:
pval = 0.05  # arbitrary
dfn = 2 - 1  # degrees of freedom numerator
dfd = med_anal.shape[0] - 2  # degrees of freedom denominator
thresh = scipy.stats.f.ppf(1 - pval, dfn=dfn, dfd=dfd)  # F distribution
print(thresh)

In [ ]:
perm_nd = np.empty([6,med_anal.shape[0]])
perm_nd[:] = np.nan
perm_nd[0,:] = np.transpose(med_anal)

In [ ]:
y = np.expand_dims(med_anal, axis = 2)
y1 = np.expand_dims(stim_anal, axis = 2)


In [ ]:
#H0 = np.empty([1000,6])
T_obs, clusters, cluster_p_values, H0 = \
    permutation_cluster_test([y, y1], n_permutations=1000,
                             threshold=None, n_jobs=None)


In [ ]:
cluster_p_values